In [0]:
!pip install sacrebleu

In [0]:
import re
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import sacrebleu
from tqdm import tqdm_notebook as tqdm
# import nltk
from nltk.translate.bleu_score import SmoothingFunction
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_file = '/content/eng-fra.txt'
test_file = '/content/testeng-fra.txt'

In [0]:
%matplotlib inline

Class Lang is to create word2index and index2word mapping dictionaries for both languages. The first 3 index-word pairs are ``{0: "SOS", 1: "EOS", 2: "PAD"}``

In [0]:
SOS_token = 0
EOS_token = 1
PAD_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "PAD"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

These functions are used to cleanse the data - remove punctuations, remove numbers and take only ASCII characters.




In [0]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [0]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    print(input_lang, output_lang, pairs)
    return input_lang, output_lang, pairs

For this particular dataset, most of the sentences are of very big lengths. Hence, we fix a big value for the MAX_LENGTH. The functions below filters out sentences which have number of words less than MAX_LENGTH




In [0]:
MAX_LENGTH = 20

# eng_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH #and \
        # p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [0]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
<__main__.Lang object at 0x7fd8af56b160> <__main__.Lang object at 0x7fd8af5568d0> [['vous savez un des plaisirs intenses du voyage et un des delices de la recherche ethnographique est la possibilite de vivre parmi ceux qui n ont pas oublie les anciennes coutumes qui ressentent encore leur passe souffler dans le vent qui le touchent dans les pierres polies par la pluie le degustent dans les feuilles ameres des plantes .', 'you know one of the intense pleasures of travel and one of the delights of ethnographic research is the opportunity to live amongst those who have not forgotten the old ways who still feel their past in the wind touch it in stones polished by rain taste it in the bitter leaves of plants .'], ['le fait de savoir que les jaguar shaman voyagent toujours au dela de la voie lactee ou que les mythes des anciens inuit resonnent encore de sens ou bien que dans l himalaya les bouddhistes continuent a rechercher le souffle du dharma c est se rappeler de la reve

In [0]:
pairs

[['bien sur nous partageons tous les memes imperatifs d adaptation .',
  'and of course we all share the same adaptive imperatives .'],
 ['nous sommes tous nes . nous apportons nos enfants dans ce monde .',
  'we re all born . we all bring our children into the world .'],
 ['nous procedons a des rites d initiations .',
  'we go through initiation rites .'],
 ['et ceci est une idee si on y reflechit qui ne peut que vous remplir d espoir .',
  'and this is an idea if you think about it can only fill you with hope .'],
 ['l ethnosphere est l heritage de l humanite .',
  'the ethnosphere is humanity s great legacy .'],
 ['et l indicateur le plus fiable est bien sur l extinction du langage .',
  'and the great indicator of that of course is language loss .'],
 ['lorsque chacun d entre vous dans cette salle est ne langues etaient pratiquees sur la planete .',
  'when each of you in this room were born there were languages spoken on the planet .'],
 ['une langue est une etincelle de l esprit 

The Seq2Seq Model
=================

We have implemented the seq2seq model below using GRUs.

The Encoder
-----------




In [0]:
BATCH_SIZE = 128
hidden_size = 512

In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        # self.embedding.weight = torch.nn.Parameter(init_embeddings)
        
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, -1, self.hidden_size)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        result = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

Attention Decoder
-----------

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        # self.embedding.weight = torch.nn.Parameter(init_embeddings)
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.n_layers)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, -1, self.hidden_size)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).transpose(0, 1)

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self, batch_size):
        result = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result



Training
========

Preparing Training Data
-----------------------





In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    indexes = indexes + [PAD_token] * (MAX_LENGTH - len(indexes)) 
    # print('indexes', indexes)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result

def tensorsFromPair(pair):
    input_variable = tensorFromSentence(input_lang, pair[0])
    target_variable = tensorFromSentence(output_lang, pair[1])
    return (input_variable, target_variable)

Training the Model
------------------
Here, we are not using teacher forsing. The variable ``use_teacher_forcing`` has been set to ``False``



In [0]:
teacher_forcing_ratio = 0.5


def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden(BATCH_SIZE)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = MAX_LENGTH
    target_length = MAX_LENGTH
    
    encoder_outputs = Variable(torch.zeros(BATCH_SIZE, max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
    
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[:, ei] = encoder_output[0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]] * BATCH_SIZE))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    decoder_hidden = encoder_hidden

    use_teacher_forcing = False #if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di]  # Teacher forcing


    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(k=1, dim=1)
            
            decoder_input = Variable(torch.LongTensor([[ni] for ni in topi[:, 0]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])

    loss.backward()    
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(BATCH_SIZE)]
        input_variable = torch.cat([pair[0] for pair in training_pairs], 1)
        target_variable = torch.cat([pair[1] for pair in training_pairs], 1)
        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            # evaluateRandomly(encoder, decoder)
    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH, beam_width=3):
    input_variable = tensorFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden(beam_width)

    input_variable = torch.cat([input_variable for i in range(beam_width)], 1)
    encoder_outputs = Variable(torch.zeros(beam_width, max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[:, ei] = encoder_output[0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]] * beam_width))  # SOS
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoder_hidden = encoder_hidden
    decoder_hidden_copy = Variable(torch.zeros(encoder1.n_layers, beam_width, hidden_size))
    decoder_hidden_copy = decoder_hidden_copy.cuda() if use_cuda else decoder_hidden_copy
    
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    probability = torch.zeros(beam_width).view(-1, 1)
    probability = probability.cuda() if use_cuda else probability
    
    prev = [[]] * max_length
    idxs = [[]] * max_length
    
    cands = []
    values = []
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
          
#       LogSoftmax
        topv, topi = decoder_output.data.topk(beam_width)
        if di == 0:
            decoder_input = Variable(torch.LongTensor([[ni] for ni in topi[0]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            probability = topv[0].view(-1, 1)
        else:
            # print('printing topi', topi)
#           Get beam_width candidate for each beam.
            topv = topv + probability
            topv, topi = topv.view(-1), topi.view(-1)

#           Select beam_width from beam_width*beam_width.
            _, topt = topv.topk(beam_width)

#           Update adn prepare for the next step.
            probability = topv[topt]
            decoder_input = Variable(torch.LongTensor([[topi[topt[k]]] for k in range(beam_width)]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            prev[di] = [topt[k] // beam_width for k in range(beam_width)]
            
#           Don't forget to prepare the corresponding hidden state.
            for i in range(beam_width):
                decoder_hidden_copy[:, i] = decoder_hidden[:, prev[di][i]]
            decoder_hidden = decoder_hidden_copy.clone()
            
        for i in range(beam_width):
            values.append([decoder_input.data[i, 0].item(), di, i])
            if decoder_input.data[i, 0] == PAD_token and probability[i] > -np.inf:
                cands.append([probability[i].item(), di, i])
                probability[i] = -np.inf
    max_proba = -500000000000000
    k = 0
    word_len = 0
    for i in range(len(cands)):
        proba = cands[i][0] 
        if(proba>max_proba):
            max_proba = proba
            k = cands[i][2] 
            word_len = cands[i][1]+1
    final_list = [output_lang.index2word[a] for a,b,c in values if c==k][:word_len]
    return final_list, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words).replace('PAD', '').replace('EOS', '')
        print('<', output_sentence)
        print('')
        bleu = sacrebleu.corpus_bleu(pair[1], output_sentence)
        print("Score=",bleu.score)        

Training and Evaluating
=======================



In [0]:
# hidden_size = 256
# encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
# trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

# # attn_decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
# # trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

encoder1 = EncoderRNN(input_lang.n_words, hidden_size, n_layers=2).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, n_layers=2, dropout_p=0.1).to(device)

In [0]:
trainIters(encoder1, attn_decoder1, 4000, print_every=50, plot_every=200, learning_rate=1e-3)
torch.save(encoder1.state_dict(), 'encoder_beam.params')
torch.save(attn_decoder1.state_dict(), 'decoder_beam.params')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0m 5s (- 7m 23s) (50 1%) 3.8685
0m 11s (- 7m 25s) (100 2%) 3.2845
0m 17s (- 7m 16s) (150 3%) 2.9296
0m 22s (- 7m 10s) (200 5%) 2.6052
0m 28s (- 7m 2s) (250 6%) 2.3334
0m 33s (- 6m 55s) (300 7%) 2.0529
0m 39s (- 6m 49s) (350 8%) 1.7944
0m 44s (- 6m 43s) (400 10%) 1.5302
0m 50s (- 6m 37s) (450 11%) 1.3503
0m 55s (- 6m 31s) (500 12%) 1.1698
1m 1s (- 6m 25s) (550 13%) 0.9958
1m 7s (- 6m 19s) (600 15%) 0.8492
1m 12s (- 6m 13s) (650 16%) 0.6874
1m 17s (- 6m 7s) (700 17%) 0.5868
1m 23s (- 6m 0s) (750 18%) 0.4828
1m 28s (- 5m 54s) (800 20%) 0.4292
1m 34s (- 5m 49s) (850 21%) 0.2932
1m 39s (- 5m 43s) (900 22%) 0.2193
1m 45s (- 5m 37s) (950 23%) 0.1926
1m 50s (- 5m 31s) (1000 25%) 0.1099
1m 56s (- 5m 25s) (1050 26%) 0.0724
2m 1s (- 5m 20s) (1100 27%) 0.0527
2m 6s (- 5m 14s) (1150 28%) 0.0397
2m 12s (- 5m 8s) (1200 30%) 0.0314
2m 17s (- 5m 3s) (1250 31%) 0.0261
2m 23s (- 4m 57s) (1300 32%) 0.0221
2m 28s (- 4m 51s) (1350 33%) 0.0190
2m 34s (- 4m 46s) (1400 35%) 0.0172
2m 39s (- 4m 40s) (1450 36%) 

In [0]:
encoder1.load_state_dict(torch.load('encoder_beam.params', map_location={'cuda:0': 'cpu'}))
attn_decoder1.load_state_dict(torch.load('decoder_beam.params', map_location={'cuda:0': 'cpu'}))

<All keys matched successfully>

In [0]:
evaluateRandomly(encoder1, attn_decoder1)

> les gens devenaient fous .
= people were going crazy .
< people were going crazy .  

Score= 100.00000000000004
> sinon va embeter quelqu un d autre aujourd hui .
= otherwise go bother somebody else today .
< otherwise go bother somebody else today .  

Score= 100.00000000000004
> et pour nathaniel la musique c est la sante mentale .
= and for nathaniel music is sanity .
< and for nathaniel music is sanity .  

Score= 100.00000000000004
> jusqu a ce que finalement j aie au telephone le biologiste en chef .
= until finally i got on the phone with the head biologist .
< until finally i got on the phone with the head biologist .  

Score= 100.00000000000004
> objectivement nous faisons en general mieux et nous nous sentons moins bien .
= and so the net result is that we do better in general objectively and we feel worse .


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


< and so the net result is that we do better in general objectively and we feel worse .  

Score= 100.00000000000004
> vous l avez peut etre lu nous avons insere des signatures .
= you may have read that we put watermarks in .
< you may have read that we put watermarks in .  

Score= 100.00000000000004
> il est impossible de de le percevoir correctement .
= there s just no way of getting it right .
< there s just no way of getting it right .  

Score= 100.00000000000004
> une productivite merveilleuse . j aime cette idee .
= blissful productivity . i love it .
< blissful productivity . i love it .  

Score= 100.00000000000004
> on en parle de facon ambivalente .
= we talk about it ambivalently .
< we talk about it ambivalently .  

Score= 100.00000000000004
> a cette epoque j aspirais a ce genre de normalite .
= i aspired to that kind of normalcy back then .
< i aspired to that kind of normalcy back then .  

Score= 100.00000000000004


Testing
=======

In [0]:
def read_test(test_file, reverse = False):
    # Read the file and split into lines
    lines = open(test_file, encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
    return pairs

In [0]:
def evaluateFully(encoder, decoder, test_file):
    evaluated = []
    pairs = read_test(test_file, reverse = True)
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    pairs0 = [p[0] for p in pairs]
    pairs1 = [p[1] for p in pairs]
    start = time.time()
    for i in tqdm(range(len(pairs0))):
        pair0 = pairs0[i]
        output_words, attentions = evaluate(encoder, decoder, pair0)
        output_words = [word for word in output_words if word not in ['PAD', 'EOS']]
        output_sentence = ' '.join(output_words)
        evaluated.append(output_sentence)
    end = time.time()
    exectime =  (end-start)/60
    print(len(evaluated))
    print(len(pairs1))
    return evaluated, pairs1, exectime
    # score = sacrebleu.corpus_bleu(evaluated, pairs1)
    # print(score)
    # for i in range(len(pairs)):
        # pair = pairs[i]
        # print('>', pair[0])
        # print('=', pair[1])
        # output_words, attentions = evaluate(encoder, decoder, pair[0])
        # output_sentence = ' '.join(output_words).replace('PAD', '').replace('EOS', '')
        # print('<', output_sentence)
        # print('')
        # bleu = sacrebleu.corpus_bleu(pair[1], output_sentence)
        # print("Score=",bleu.score) 

In [0]:
pairs_test = read_test('testeng-fra.txt', reverse=True)
evaluated, pairs1, exectime = evaluateFully(encoder1, attn_decoder1, test_file)
print('execution time is', exectime, 'minutes')

Counted words:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



1164
1164
execution time is 0.7584611694018046 minutes


In [0]:
bleu = sacrebleu.corpus_bleu(evaluated, [pairs1]) 
print(bleu.score)

1.4354195909500966
